In [ ]:
# Required Libraries 
import numpy as np 
import os 

# Keras Libraries 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


# My libraries 
from training import training_main
from unet3d import get_unet3D_multi 
from cunet3d import get_Cunet3D_mulit
from training import training_main 
from testing3D import testing_parallel
from unetplusplus3d import get_unetpp3D_multi


print('-'*100)
print('Loading Data...')
print('-'*100)

dpath = # Enter path here  

shift = np.load(dpath+'scar_mean.npy')

imgs_ext = # Load your images here 
segs_ext = np.zeros(shape=imgs_ext.shape)


path1 = # Enter Weight Path 

# Create the path 
if not os.path.exists(path1):
    os.mkdir(path1)

print('-'*100)
print('Defining Model...')
print('-'*100)

# Image Parameters 
img_rows = 256
img_cols = 256
slices = 16 


# Model Parameters 
classes = 3 
layers = 4
min_convs = 4
kernel = (3,3,3)
lossfun = 'wcce_kld' # Need to write something for this later - might clean up model code as well 

# Hyperparameters 
decay = 0.98
learning_rate = 1e-3 
batches = 4
epochs_total = 100
epochs_batch = 5 
weights_init = (1,20,1000) 

strategy = tf.distribute.MirroredStrategy()

train_loss = np.zeros((epochs_total,))
train_dice = np.zeros((epochs_total,))
train_myo = np.zeros((epochs_total,))

val_loss = np.zeros((epochs_total,))
val_dice = np.zeros((epochs_total,))
val_myo = np.zeros((epochs_total,))

# Generate the model - either U-Net, Cascaded U-Net, or U-Net++ 
with strategy.scope():
    model = get_unet3D_multi(img_rows, img_cols, slices, classes, layers, min_convs, kernel, learning_rate, lossfun, weights_init)
model.summary()


old_model_weight_path = path1 + 'final_weights.h5'
model.load_weights(old_model_weight_path)

print('-'*100)
print('Validating Model...')
print('-'*100)

savepath = path1+'validation_external/'

# Create the path 
if not os.path.exists(savepath):
    os.mkdir(savepath)

gpus = 2 

# Testing model on validation set only 
preds, contours = testing_parallel(savepath, model, imgs_ext, segs_ext, gpus, classes, shift)